In [ ]:
import psycopg2
import pandas as pd
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
import os
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
POSTGRESQL_HOST=os.getenv("POSTGRESQL_HOST")
POSTGRESQL_PORT=os.getenv("POSTGRESQL_PORT")
POSTGRESQL_DATABASE=os.getenv("POSTGRESQL_DATABASE")
POSTGRESQL_USER=os.getenv("POSTGRESQL_USER")
POSTGRESQL_PASSWORD=os.getenv("POSTGRESQL_PASSWORD")

print(POSTGRESQL_HOST,POSTGRESQL_PORT,POSTGRESQL_DATABASE,POSTGRESQL_USER)

conn = psycopg2.connect(
    host=POSTGRESQL_HOST,
    port=POSTGRESQL_PORT,
    dbname=POSTGRESQL_DATABASE,
    user=POSTGRESQL_USER,
    password=POSTGRESQL_PASSWORD
)


10.222.117.105 5432 token_tracer zhuyufei


In [6]:
# 选择一个 token 的表，比如 "crv_transactions"
df = pd.read_sql(
    """
    SELECT *
    FROM crv_transactions
    limit 10
    """,
    conn
)

C:\Users\zhuyufei\AppData\Local\Temp\ipykernel_22040\2148033441.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [ ]:
# ========== 节点映射 ==========
# 每一笔交易就是一个节点
# 用 transaction_hash 来标识唯一节点
tx_to_idx = {tx: i for i, tx in enumerate(df["transaction_hash"].unique())}

# ========== 节点特征 ==========
# 选取数值型字段作为节点特征
node_features = df[[
    "block_number", 
    "transaction_index", 
    "gas_used", 
    "cumulative_gas_used", 
    "gas_price"
]].fillna(0)

scaler = StandardScaler()
node_features = torch.tensor(
    scaler.fit_transform(node_features),
    dtype=torch.float
)

# ========== 边 ==========
# 这里暂时没有规则，就先建一个空的 edge_index
# 后面你可以根据规则去生成 edge_index
edge_index = torch.empty((2, 0), dtype=torch.long)  

# ========== 边特征 ==========
# 由于还没有边，所以边特征先为空，后面补充transaction之间连边的规则，相关的transaction之间连一条边
edge_features = torch.empty((0, 1), dtype=torch.float)

# ========== 构建图数据 ==========
data = Data(
    x=node_features,        # 节点特征 (N, F)
    edge_index=edge_index,  # 边 (2, E)
    edge_attr=edge_features # 边特征 (E, F)
)

print(data)
# 打印前5个节点及其特征
for i in range(min(5, data.num_nodes)):
    tx_hash = df.iloc[i]["transaction_hash"]
    features = data.x[i].tolist()
    print(f"Node {i} (tx_hash={tx_hash}): {features}")


Data(x=[10, 5], edge_index=[2, 0], edge_attr=[0, 1])
Node 0 (tx_hash=0xcac4eadcf267993d1bdc5422e963db6fa22410fef666009a015d8661e8b51652): [-1.4439897537231445, 0.5111734867095947, 0.40480664372444153, 1.2119958400726318, -0.6079502105712891]
Node 1 (tx_hash=0xdf7632fe3dc93ec212708c5401c82a1ede1a3a84a9ac858f7df2359344f16fd1): [-0.9282791018486023, -0.8774945139884949, -1.0624866485595703, -0.6498267650604248, -0.5187986493110657]
Node 2 (tx_hash=0x0ca5f025586b6a824ecffeed8c07f0b901ac7305149c29e9f86d9f5b738e2179): [-0.9282791018486023, -0.13527540862560272, -0.4274420738220215, 0.6909891963005066, -0.6270541548728943]
Node 3 (tx_hash=0x04ceb7f71932d6ce45fa8437a5cd2713225f9849d81b7076ff61bb95196054ce): [-0.41256850957870483, -1.021149754524231, -1.0624866485595703, -1.0965148210525513, -0.25134381651878357]
Node 4 (tx_hash=0x818df4c2ddc829716dd95b9771604f5eb8f13b94f5f15481abc41b160b488156): [-0.41256850957870483, 0.7625702619552612, 1.3402550220489502, 0.8721851706504822, -0.6334221363067